In [ ]:
#importing the library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [ ]:
#collecting data
dataset = pd.read_csv('../input/datasetbohm/BOMH.csv')
dataset.head()

In [ ]:
dataset.isna().any()

In [ ]:
dataset.info()

In [ ]:
dataset['Close Price'].plot(figsize=(16,6))

In [ ]:
dataset2=dataset.reset_index()['Close Price']
dataset2[:]

In [ ]:
#LSTM are sensitive to the scale of the data, so we apply minmax scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
dataset2=scaler.fit_transform(np.array(dataset2).reshape(-1,1))

In [ ]:
print(dataset2)

In [ ]:
#splitting data set into train and test split
training_size=int(len(dataset2)*0.65)
test_size=len(dataset2)-training_size
train_data=dataset2[0:training_size , :]
test_data=dataset2[training_size:len(dataset2),:1]

In [ ]:
print(training_size , test_size)

In [ ]:
import numpy
#convert an array of value into a dataset matrix
def create_dataset(dset,time_step=1):
    dataX , dataY = [], []
    for i in range(len(dset)-time_step-1):
        a = dset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dset[i + time_step,0])
    return numpy.array(dataX) , numpy.array(dataY)

In [ ]:
#reshape into X=t, t+1,t+2,t+3 nad Y=t+4
time_step = 100
X_train , Y_train = create_dataset(train_data, time_step)
X_test , Y_test = create_dataset(test_data, time_step)

In [ ]:
print(X_train, Y_train)


In [ ]:
print(X_test, Y_test)

In [ ]:
# reshape input to be [sample , time_steps  , features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0] , X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
#create the stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error' , optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,Y_train, validation_data=(X_test,Y_test),epochs=100,batch_size=64,verbose=1)

In [ ]:
import tensorflow as tf

In [ ]:
#PREDICTION AND CHECK PERFORMANCE METRICS
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
# transform to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)


In [ ]:
#calculating RSME performance
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(Y_train,train_predict))

In [ ]:
#Test Data RSME
math.sqrt(mean_squared_error(Y_test,test_predict))

In [ ]:
len(test_data)

In [ ]:
X_input=test_data[618:].reshape(1,-1)
X_input.shape

In [ ]:
temp_input=list(X_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input

In [ ]:
# demonstrate prediction for next 30 days
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    if(len(temp_input)>100):
        #print(temp_input)
        X_input=np.array(temp_input[1:])
        print("{}day input {}".format(i,X_input))
        X_input=X_input.reshape(1,-1)
        X_input=X_input.reshape(1,n_steps,1)
        #print(x_input)
        yhat=model.predict(X_input,verbose=0)
        print("{} day out {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        X_input = X_input.reshape((1,n_steps,1))
        yhat = model.predict(X_input , verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
        
print(lst_output)
        
        

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(dataset2)

In [ ]:
dataset3=dataset2.tolist()
dataset3.extend(lst_output)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(dataset2[1949:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
dataset3=dataset2.tolist()
dataset3.extend(lst_output)
plt.plot(dataset3[2049:])